# LangChain Agents: A Deep Dive

**Objective:** This notebook provides a detailed walkthrough of building powerful, single agents using LangChain. While the course's Day 5 lab introduced the basics, this session focuses on the internal mechanics, advanced tool creation, and state management to give you a deeper understanding of how to build and debug robust LangChain agents.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Concepts Covered:**
1.  **Review of Agent Fundamentals:** A quick recap of the `AgentExecutor` and the ReAct (Reason+Act) loop.
2.  **Custom Tools in Detail:** Mastering the `@tool` decorator and understanding why a good docstring is the most critical part of a tool.
3.  **Agent State & Memory:** Giving your agent conversational memory to handle multi-turn interactions.
4.  **Debugging Agent Behavior:** Using `verbose=True` to interpret the agent's thought process and fix issues.

## 1. Setup

First, we'll install the necessary libraries and set up our environment by loading the required API keys.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    print("ERROR: API keys not found. Please check your .env file.")

## 2. Review of Agent Fundamentals

Let's quickly recap the core components of a LangChain agent from the Day 5 lab. An agent consists of:

- **LLM:** The language model that acts as the agent's "brain."
- **Tools:** Functions the agent can call to interact with the outside world (e.g., search, calculators, file readers).
- **Prompt Template:** A template that structures the input to the LLM, telling it its persona, the tools it has, and the user's request. It critically includes an `agent_scratchpad` for the ReAct loop.
- **Agent Executor:** The runtime environment that orchestrates the **Thought -> Action -> Observation** loop.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatOpenAI(model="gpt-4o")
tools = [TavilySearchResults(max_results=1)]
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("--- Recap: Agent with Web Search ---")
agent_executor.invoke({"input": "What is the market cap of Apple Inc. as of September 3 2025?"})

C:\Users\labadmin\AppData\Local\Temp\2\ipykernel_6592\1963754524.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tools = [TavilySearchResults(max_results=1)]


--- Recap: Agent with Web Search ---


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Apple Inc. market cap September 3 2025'}`


[{'title': 'Apple Inc. (AAPL) Stock Historical Prices & Data - Yahoo Finance', 'url': 'https://finance.yahoo.com/quote/AAPL/history/', 'content': '| Sep 4, 2025 | 238.45 | 239.90 | 236.74 | 239.78 | 239.78 | 47,549,400 |\n| Sep 3, 2025 | 237.21 | 238.85 | 234.36 | 238.47 | 238.47 | 66,427,800 |\n| Sep 2, 2025 | 229.25 | 230.85 | 226.97 | 229.72 | 229.72 | 44,075,600 |\n| Aug 29, 2025 | 232.51 | 233.38 | 231.37 | 232.14 | 232.14 | 39,418,400 |\n| Aug 28, 2025 | 230.82 | 233.41 | 229.34 | 232.56 | 232.56 | 38,074,700 |\n| Aug 27, 2025 | 228.61 | 230.90 | 228.26 | 230.49 | 230.49 | 31,259,500 | [...] | Sep 12, 2025 | 229.22 | 234.51 | 229.02 | 234.07 | 234.07 | 55,824,200 |\n| Sep 11, 2025 | 226.88 | 230.45 | 226.65 | 230.03 | 230.03 | 50,208,600 |\n| Sep 10, 2025 | 232.19 | 232.42 | 225.95 | 226.79 | 226.79 | 83,

{'input': 'What is the market cap of Apple Inc. as of September 3 2025?',
 'output': "As of September 3, 2025, Apple Inc.'s stock closed at $238.47. To determine the market capitalization, we would typically multiply the closing stock price by the total number of outstanding shares. Since the total number of outstanding shares is not provided in this data, the precise market cap cannot be calculated directly from this information. You would need the number of outstanding shares to find the exact market cap."}

## 3. Custom Tools in Detail

The real power of LangChain agents comes from giving them custom tools to interact with your specific data or APIs. The `@tool` decorator is the easiest way to do this.

**The Golden Rule of Tools:** The agent **does not see your Python code**. It only sees the function name, its arguments (and their type hints), and the **docstring**. A clear, descriptive docstring is the most important piece of prompt engineering you can do for an agent.

In [3]:
from langchain.tools import tool
import json

# A simple custom tool
@tool
def simple_calculator(expression: str) -> str:
    """Calculates the result of a simple mathematical expression (e.g., '2+2', '10*5')."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

# A tool that interacts with the local file system
# In a real app, ensure this is secure and sandboxed!
@tool
def read_file_content(filename: str) -> str:
    """Reads the content of a specified file. Use this to get information from local files."""
    try:
        with open(filename, 'r') as f:
            return f.read()
    except FileNotFoundError:
        return f"Error: File '{filename}' not found."

# Create a new agent with multiple custom tools
custom_tools = [simple_calculator, read_file_content]
custom_agent = create_tool_calling_agent(llm, custom_tools, prompt)
custom_executor = AgentExecutor(agent=custom_agent, tools=custom_tools, verbose=True)

print("--- Testing the Calculator Tool ---")
custom_executor.invoke({"input": "What is 125 multiplied by 8?"})

# Let's create a dummy file for the next test
with open("project_status.txt", "w") as f:
    f.write("The project is on track. The next milestone is the agent evaluation phase.")

print("\n--- Testing the File Reader Tool ---")
custom_executor.invoke({"input": "What is the status of the project based on the 'project_status.txt' file?"})

--- Testing the Calculator Tool ---


> Entering new AgentExecutor chain...

Invoking: `simple_calculator` with `{'expression': '125*8'}`


1000125 multiplied by 8 is 1000.

> Finished chain.

--- Testing the File Reader Tool ---


> Entering new AgentExecutor chain...

Invoking: `read_file_content` with `{'filename': 'project_status.txt'}`


The project is on track. The next milestone is the agent evaluation phase.The project is on track. The next milestone is the agent evaluation phase.

> Finished chain.


{'input': "What is the status of the project based on the 'project_status.txt' file?",
 'output': 'The project is on track. The next milestone is the agent evaluation phase.'}

## 4. Agent State & Memory

By default, the `AgentExecutor` is stateless. Each call is independent. To have a conversation, we need to add a memory component. The easiest way is to include placeholders for chat history in the prompt and manage the history manually.

LangChain's Expression Language (LCEL) makes this more manageable.

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# 1. Create a new prompt that includes a placeholder for chat history
memory_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You have access to a calculator."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 2. Create the agent with the new prompt
memory_tools = [simple_calculator]
memory_agent = create_tool_calling_agent(llm, memory_tools, memory_prompt)
memory_executor = AgentExecutor(agent=memory_agent, tools=memory_tools, verbose=True)

# 3. Manage the chat history manually
chat_history = []

print("--- Conversation Turn 1 ---")
input1 = "My budget is 500. I spent 120 on a book. How much is left?"
result1 = memory_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend([
    HumanMessage(content=input1),
    AIMessage(content=result1["output"]),
])
print(f"\nAI: {result1['output']}")

print("\n--- Conversation Turn 2 (with memory) ---")
input2 = "If I spend another 55, what is the remaining amount?"
result2 = memory_executor.invoke({"input": input2, "chat_history": chat_history})
print(f"\nAI: {result2['output']}")

--- Conversation Turn 1 ---


> Entering new AgentExecutor chain...

Invoking: `simple_calculator` with `{'expression': '500-120'}`


380You have 380 left after spending 120 on a book from your budget of 500.

> Finished chain.

AI: You have 380 left after spending 120 on a book from your budget of 500.

--- Conversation Turn 2 (with memory) ---


> Entering new AgentExecutor chain...

Invoking: `simple_calculator` with `{'expression': '380 - 55'}`


325If you spend another 55, you will have 325 remaining.

> Finished chain.

AI: If you spend another 55, you will have 325 remaining.


## 5. Debugging Agent Behavior

The most important tool for debugging an agent is `verbose=True` in the `AgentExecutor`. This prints out the agent's internal monologue, showing you the **Thought -> Action -> Observation** loop in real-time.

When an agent fails, look at the verbose output:
-   **Thought:** What was the agent thinking? Did it misunderstand the goal?
-   **Action:** Which tool did it choose? Were the parameters correct? If it didn't choose a tool when it should have, the tool's docstring might be unclear.
-   **Observation:** What was the output of the tool? Was it an error? Was the format unexpected?

Most agent failures can be traced back to a poorly worded prompt or an unclear tool docstring.

## Lab Conclusion

In this deep dive, you've reinforced your understanding of LangChain agents. You've seen how to create powerful, specialized agents by giving them custom tools and how to add memory for conversational context. Most importantly, you've learned how to debug agentic behavior by analyzing the ReAct loop.

**Key Takeaways:**
- An agent's capabilities are defined by its tools.
- A tool's docstring is its most important feature.
- Conversational memory requires explicitly managing and passing chat history.
- `verbose=True` is your best friend for debugging.